In [1]:
# Ir um por vez

# !gdown '1pMVzpQnOethGJ8b94IJddLOnvfo5zJNV'
# !unzip "/content/RawFold0.zip" -d "/content/Fold0"
# !rm -rf '/content/RawFold0.zip'

!gdown '1RGw9a1Qrv5_5QRwOpH4nl55GukjgUNHf'
!unzip "/content/RawFold1.zip" -d "/content/RawFold1"
!rm -rf '/content/RawFold1.zip'

# !gdown '1TLnuaH52b3La5Pz3EVRQDIe2yUwPxon_'
# !unzip "/content/Fold2.zip" -d "/content/Fold2"
# !rm -rf '/content/Fold2.zip'

# !gdown '1ED6MMqHo5MuU8IQepMHsZqVYN_bwbVFY'
# !unzip "/content/Fold3.zip" -d "/content/Fold3"
# !rm -rf '/content/Fold3.zip'

# !gdown '141uHRgxuPycaLNfItfSaK4Uyt4BYG9N1'
# !unzip "/content/Fold4.zip" -d "/content/Fold4"
# !rm -rf '/content/Fold4.zip'

Downloading...
From: https://drive.google.com/uc?id=1RGw9a1Qrv5_5QRwOpH4nl55GukjgUNHf
To: /content/RawFold1.zip
100% 1.55G/1.55G [00:10<00:00, 146MB/s]
Archive:  /content/RawFold1.zip
  inflating: /content/RawFold1/Train_Fold1.npy  
  inflating: /content/RawFold1/Test_Fold1.npy  


In [2]:
import numpy as np
import scipy as sp
from scipy import signal
from scipy.signal import hilbert
import matplotlib.pyplot as plt

In [3]:
def segmenta(sinal, passo, dimensoes):
  #Inputs
  #- sinal: sinal a ser segmentado
  #- passo: quantas amostras irá avançar de um segmento para outro
  #- dimensoes: Dimensões esperada do output

  #Output
  #- segmentado: Sinal segmentado em segmentos de 2*passo de duração e com
  # overlap de 1 passo entre eles

  segmentado = np.zeros(dimensoes)
  cont = 0
  for linha in range(sinal.shape[0]):
      sinal_s = sinal[linha,:,:]
      for i in range(1000000):
          if ~np.any(sinal_s[i*passo:(i+2)*passo,:]) or sinal_s[i*passo:(i+2)*passo,:].shape[0] != int(2*passo):
              break
          segmentado[cont,:,:] = sinal_s[i*passo:(i+2)*passo,:]
          cont = cont+1
  return segmentado

def Features(sinal,fs=500):
  #Inputs
  #- sinal: sinal de onde serão tiradas as características
  #- fs: frequência de amostragem do sinal

  #Output
  #- sig_features: 

  sig_features = np.zeros((256,20))

  # A extração de características é feita canal por canal
  for i in range(20):
    sinal_c = sinal[:,i]

    # Encontra a frequência instantanea do canal utilizando a transformada de Hilbert
    analytic_signal = hilbert(sinal_c)
    instantaneous_phase = np.unwrap(np.angle(analytic_signal))
    instantaneous_frequency = np.abs((np.diff(instantaneous_phase) / (2.0*np.pi) * fs))
    instantaneous_frequency = instantaneous_frequency.reshape((-1,1))
    

    # Determina a densidade do espectro de potência do canal
    f, psd = signal.welch(sinal_c, 500)

    # f, psd = sp.signal.periodogram(sinal_c,freq=250)

    # alfa = (f>=8) & (f<12)
    # alfa = psd[alfa].sum().reshape((-1,1))

    # beta = (f>=12) & (f<30)
    # beta = psd[beta].sum().reshape((-1,1))
    
    # gamma = (f>=30)
    # gamma = psd[gamma].sum().reshape((-1,1))

    # delta = (f>=2) & (f<4)
    # delta = psd[delta].sum().reshape((-1,1))

    # theta = (f>=4) & (f<8)
    # theta = psd[theta].sum().reshape((-1,1))

    alfa = np.trapz(y = psd[4:6], x = f[4:6]).reshape((-1,1))
    beta = np.trapz(y = psd[6:15], x = f[6:15]).reshape((-1,1))
    gamma = np.trapz(y = psd[15:50], x = f[15:50]).reshape((-1,1))
    delta = np.trapz(y = psd[0:2], x = f[0:2]).reshape((-1,1))
    theta = np.trapz(y = psd[2:4], x = f[2:4]).reshape((-1,1))

    sig_mean = np.mean(sinal_c).reshape((-1,1))

    sig_std = np.std(sinal_c).reshape((-1,1))
    


    # # Determina a entropia dispersa do sinal (dim = 1, 1) 
    # dist, phi = EH.DispEn(sinal_c)
    # dist = dist.reshape((-1,1))
    # phi = phi.reshape((-1,1))

    # # Determina a entropia aproximada do sinal (dim = 3,4)
    # app, phi_app = EH.ApEn(sinal_c)
    # app = app.reshape((-1,1))
    # phi_app = phi_app.reshape((-1,1))

    # feat = np.concatenate((instantaneous_frequency,psd,sig_mean,sig_std,dist,phi,app,phi_app)).reshape((-1,1))
    feat = np.concatenate((instantaneous_frequency,delta,theta,alfa,beta,gamma,sig_mean,sig_std)).reshape((-1,1))

    

    sig_features[:,i] = feat[:,0]
  
  return sig_features

In [4]:
train = np.load('/content/RawFold1/Train_Fold1.npy')
test = np.load('/content/RawFold1/Test_Fold1.npy')

np.random.seed(42)

np.random.shuffle(train)
np.random.shuffle(test)

In [5]:
train = segmenta(train,125,(54984,250,22))
test = segmenta(test,125,(20935,250,22))

In [6]:
x_train, y_train = train[:,:,:20], train[:,:,-1]
y_train = np.mean(y_train, axis=1,dtype=np.int16).reshape((-1,1))

data_train = np.zeros((x_train.shape[0],256,20))
for segm in range(x_train.shape[0]):
    data_train[segm,:,:] = Features(x_train[segm,:,:])

/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 250, using nperseg = 250
  .format(nperseg, input_length))


In [7]:
x_test, y_test = test[:,:,:20], test[:,:,-1]
y_test = np.mean(y_test, axis=1,dtype=np.int16).reshape((-1,1))

data_test = np.zeros((x_test.shape[0],256,20))
for segm in range(x_test.shape[0]):
    data_test[segm,:,:] = Features(x_test[segm,:,:])

In [8]:
np.save('XTrain_Fold0',data_train)
np.save('XTest_Fold0',data_test)

np.save('YTrain_Fold0',y_train)
np.save('YTest_Fold0',y_test)